# Deelmodel Steenbergerloop

In [1]:
import hydrotools
from pathlib import Path
import geopandas as gpd
import pandas as pd
from delft3dfmpy import DFlowFMModel, DFlowFMWriter

In [2]:
dimr_path = r"c:\Program Files (x86)\Deltares\D-HYDRO Suite 1D2D (Beta) (0.9.7.52006)\plugins\DeltaShell.Dimr\"kernels\x64\dimr\scripts\run_dimr.bat"

hydamo = hydrotools.load_model(Path(r"./hydamo_model/boezemmodel_v4.pickle"))
shp_file = r".\shp\steenbergerloop.shp"
geometry = gpd.read_file(shp_file).loc[0]['geometry']
hydamo = hydrotools.filter_model(hydamo, geometry=geometry)
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/steenbergerloop"))
principe_profielen_df = pd.read_excel(r".\xlsx\principe_profielen.xlsx", index_col=0)

## Converteren naar DFM

### Aanmaken dfm-klasse

In [3]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [4]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

2021-02-25 13:41:12,127 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


### Aanmaken 1d netwerk

In [5]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=100.0, seperate_structures=True)

2021-02-25 13:41:12,280 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.
2021-02-25 13:41:12,287 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000211 at: [0.0, 216.9435, 221.8745, 228.02550000000002, 235.7845, 241.2600977488463], due to the structures at [-0.001, 215.157, 218.73, 225.019, 231.032, 240.537, 241.2610977488463].
2021-02-25 13:41:12,294 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000494 at: [0.0, 40.601, 103.154, 141.216, 159.891, 202.399, 366.126, 517.109, 579.6005, 647.528, 681.4304999999999, 685.625, 736.38, 786.4423733018544], due to the structures at [-0.001, 0.5, 80.702, 125.606, 156.826, 162.956, 241.842, 338.468, 393.784, 490.151, 544.067, 615.134, 679.922, 682.939, 688.311, 784.449, 786.4433733018544].
2021-02-25 13:41:12,297 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002236 at: [0.0, 409.8995, 455.097191363579], due to the structures at [-0.001, 395

### Toevoegen dwarsprofielen

We kennen profielen toe, waar we profielen missen vullen we deze aan met principeprofielen.

In [6]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:
    print("adding trapezium profiles on branches with missing crosssections.")
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_df)

2021-02-25 13:41:12,376 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 20.
2021-02-25 13:41:12,415 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 9.
No parametrised crossections available for branches.
2021-02-25 13:41:12,417 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 10.
2021-02-25 13:41:12,439 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 10.
2021-02-25 13:41:12,485 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.
9 branches are still missing a cross section.
0 structures are still missing a cross section.
adding trapezium profiles on branches with missing crosssections.


C:\Anaconda3\envs\delft3dfmpy\lib\site-packages\delft3dfmpy\core\dfm.py:449: RuntimeWarning: invalid value encountered in double_scalars
  levels = f'0 {(maximumflowwidth - bottomwidth) / (2 * slope):.2f}'


### Wegschrijven model

In [7]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 144.0 * 1 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir="modellen\steenbergerloop", name="steenbergerloop")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r".modellen\steenbergerloop\fm\steenbergerloop.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven
